In [150]:
import os
import pandas as pd
import pandas_gbq as pd_gbq
import numpy as np

In [120]:
#FIREBASE
from firebase_admin import credentials,firestore,initialize_app
from datetime import datetime
from google.cloud import bigquery

In [3]:
filename=os.environ['FIREBASE_FILENAME']
#OPEN CONECTION TO FIREBASE
cred = credentials.Certificate(filename)
firebase = initialize_app(cred)

db = firestore.client()

In [4]:
OAN_account = "5Tv2u4n8BReebmKUNIuN"

In [282]:
''' donors = db.collection('donors'
                       ).where(u'context.account', u'==',OAN_account
                              ).stream() '''
                              
donors = db.collection('donors'
                       ).stream()

In [283]:
l_donors = []
for donorF in donors:
    #print(payout)
    l_donors.append(donorF.to_dict())


In [284]:
df_donors = pd.DataFrame(l_donors)

In [285]:
df_recurrentConfig = df_donors["recurrentConfig"].apply(pd.Series)
df_recurrentConfig = df_recurrentConfig.drop(columns=[0])

In [286]:
mapper = {"targetAccount": "recurrentConfigTargetAccount",
          "concept": "recurrentConfigConcept",
          "dayOfTheMonth":"recurrentConfigDayOfTheMonth",
          "amount":"recurrentConfigAmount"}

In [287]:
df_recurrentConfig = df_recurrentConfig.rename(mapper,axis=1)

In [288]:
df_donors = df_donors.join(df_donors["context"].apply(pd.Series))
df_donors = df_donors.join(df_recurrentConfig)

In [289]:
df_donors = df_donors.drop(columns=["context"])
df_donors = df_donors.drop(columns=["recurrentConfig"])
df_donors = df_donors.drop(columns=["customFields"])

In [290]:
queryDonors = '''
            SELECT *
            FROM `oan-miong.firestore_export.donorsView`
            '''
df_donorsGbq = pd_gbq.read_gbq(queryDonors)

/home/daniel_alfaro/.pyenv/versions/3.10.16/envs/defaultenv/lib/python3.10/site-packages/google/cloud/bigquery/table.py:2379: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(


In [291]:
df_donorsGbq.shape

(235, 26)

In [292]:
#new donoors
df_join_new = pd.merge(df_donors,df_donorsGbq['id'],how="left", on="id", indicator=True)
df_new_donors = df_join_new[df_join_new['_merge'] == 'left_only'].copy()
df_new_donors['action'] = 'created'
df_new_donors['category'] = None
df_new_donors['timestamp'] = datetime.now().isoformat()
df_new_donors = df_new_donors.drop(columns=["_merge"])
df_new_donors.shape

(0, 26)

In [300]:
#modified donoors
df_donors_not_new = df_join_new[df_join_new['_merge'] == 'both'].copy()
df_donors_not_new = df_donors_not_new.drop(columns=["_merge"])
df_merged = pd.merge(df_donors_not_new,
                     df_donorsGbq[['id','lastUpdateAt']],
                    how = 'inner',
                    on="id",
                    suffixes=("", "_gbq")
)
# Filtrar registros donde 'lastUpdateAt_not_new' es más reciente
updated_records = df_merged[
    (df_merged['lastUpdateAt'] > df_merged['lastUpdateAt_gbq']) |
    (df_merged['lastUpdateAt_gbq'].isna() & ~df_merged['lastUpdateAt'].isna())
]

# Selecciona solo las columnas de df_donors_not_new
updated_records = updated_records[df_donors_not_new.columns]
updated_records['action'] = 'updated'
updated_records['category'] = None
updated_records['timestamp'] = datetime.now().isoformat()
updated_records.shape

(1, 26)

In [301]:
df_new_donorsGbq = pd.concat([df_donorsGbq,df_new_donors,updated_records])

/tmp/ipykernel_24993/1843257094.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_new_donorsGbq = pd.concat([df_donorsGbq,df_new_donors,updated_records])


In [302]:
print([df_donorsGbq.shape,df_new_donors.shape,updated_records.shape])

[(235, 26), (0, 26), (1, 26)]


In [303]:
df_new_donorsGbq["recurrentConfigAmount"] = df_new_donorsGbq["recurrentConfigAmount"].astype(float)
df_new_donorsGbq["percentage"] = df_new_donorsGbq["percentage"].astype(float)
df_new_donorsGbq["isRecurrent"] = df_new_donorsGbq["isRecurrent"].fillna(False).astype(bool)
df_new_donorsGbq["recurrentConfigDayOfTheMonth"] = df_new_donorsGbq["recurrentConfigDayOfTheMonth"].apply(lambda x: int(x) if x>0 else None)

/tmp/ipykernel_24993/1997971792.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_new_donorsGbq["isRecurrent"] = df_new_donorsGbq["isRecurrent"].fillna(False).astype(bool)


In [304]:
# Ajusta el DataFrame para cumplir con el esquema
df_new_donorsGbq["tags"] = df_new_donorsGbq["tags"].apply(
    lambda x: x.tolist() if isinstance(x, np.ndarray) else x
)
df_new_donorsGbq["tags"] = df_new_donorsGbq["tags"].apply(
    lambda x: x if isinstance(x, list) else [x]
)

In [317]:
df_merged.columns

Index(['key', 'lastName', 'province', 'tags', 'nature', 'region', 'percentage',
       'dni', 'isRecurrent', 'email', 'name', 'partner', 'lastUpdateBy',
       'createdBy', 'id', 'account', 'createdAt', 'createdByType',
       'lastUpdateAt', 'recurrentConfigTargetAccount',
       'recurrentConfigConcept', 'recurrentConfigDayOfTheMonth',
       'recurrentConfigAmount', 'lastUpdateAt_gbq'],
      dtype='object')

In [319]:
orden = { "id": "0OORMP8HfBajJxUOa3j3",
  "action": "created",
  "timestamp": "2024-12-12T17:23:45.748Z",
  "account": "5Tv2u4n8BReebmKUNIuN",
  "category": None,
  "createdByType": "system",
  "createdBy": "oXJJEfAEPxFYtdJ2pnaU",
  "lastUpdateBy": "oXJJEfAEPxFYtdJ2pnaU",
  "createdAt": "2024-01-05T12:22:40.401Z",
  "lastUpdateAt": "2024-01-05T12:22:40.401Z",
  "name": "Antonia",
  "lastName": "Bonilla Jiménez",
  "partner": None,
  "email": "gaiajimenezescritora@gmail.com",
  "dni": "",
  "region": "málaga",
  "province": "Málaga",
  "key": "A",
  "percentage": None,
  "nature": "naturalPerson",
  "tags": [],
  "isRecurrent": "false",
  "recurrentConfigAmount": None,
  "recurrentConfigTargetAccount": None,
  "recurrentConfigDayOfTheMonth": None,
  "recurrentConfigConcept": None
}

In [324]:
df_new_donorsGbq = df_new_donorsGbq[list(orden.keys())]

In [326]:
# Configura el cliente de BigQuery
client = bigquery.Client(project="oan-miong")

# Define la tabla destino
table_id = "oan-miong.firestore_export.donors"

table = client.get_table(table_id)

schema = [{"name": schema_field.name, "type": schema_field.field_type, "mode": schema_field.mode} for schema_field in table.schema]

# Configura el job de carga
job_config = bigquery.LoadJobConfig(
    schema=schema,
    write_disposition="WRITE_TRUNCATE"  # Reemplaza la tabla si ya existe
)

# Sube el DataFrame a BigQuery
job = client.load_table_from_dataframe(df_new_donorsGbq, table_id, job_config=job_config)

# Espera a que termine el job
job.result()

print(f"Tabla cargada exitosamente a {table_id}.")

Tabla cargada exitosamente a oan-miong.firestore_export.donors.
